### How to merge the delta of 2 rows, one in each row

In [1]:
import pandas as pd

rows = [
 ('a', 5),
 ('b', 3),
 ('c', 2),
 ('d', 1)
]

df = pd.DataFrame(data=rows, columns=['id', 'score'])

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

from pyspark import SparkContext, SQLContext

from pyspark.sql.types import *

mySchema = StructType([ StructField("id", StringType(), True)\
                       ,StructField("score", LongType(), True)])

sc = SparkSession.builder.appName('testjean').getOrCreate()
sqlCtx = SQLContext(sc)
faketable = sqlCtx.createDataFrame(df, schema=mySchema)
faketable.createOrReplaceTempView('faketable')



## A solution is to use sql window functions

In [31]:
from pyspark.sql.functions import monotonically_increasing_id, lag, lead, when, isnull
from pyspark.sql.window import Window

faketable = faketable.withColumn('my_count', monotonically_increasing_id())

my_window = Window.partitionBy().orderBy("my_count")

faketable = faketable.withColumn("prev_score", lead(faketable.score).over(my_window))

## this line keeps every second line
faketable = faketable.filter(faketable.my_count % 2 == 0)

faketable.withColumn("delta", when(isnull(faketable.score - faketable.prev_score), 0).otherwise(faketable.score - faketable.prev_score)).show()






+---+-----+-----+----------+--------+-----+
| id|score|count|prev_score|my_count|delta|
+---+-----+-----+----------+--------+-----+
|  a|    5|    0|         3|       0|    2|
|  c|    2|    2|         1|       2|    1|
+---+-----+-----+----------+--------+-----+



In [4]:
#expected output
result = [
 ['a', 'b', 2],
 ['c', 'd', 1],
]

